In [1]:
!pip install -q pandas numpy scikit-learn tqdm
!pip install -q sentence-transformers
!pip install -q faiss-cpu
!pip install -q gradio requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 33.2 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import ast
from tqdm import tqdm
from collections import deque
import requests
from sentence_transformers import SentenceTransformer
from sklearn.preprocessing import normalize
import faiss
import gradio as gr

In [3]:
movies = pd.read_csv("tmdb_5000_movies.csv", engine='python', on_bad_lines='skip')
credits = pd.read_csv("tmdb_5000_credits.csv", engine='python', on_bad_lines='skip')

movies = movies.merge(credits, on="title")

if 'id_x' in movies.columns:
    movies = movies.rename(columns={'id_x': 'id'})
elif 'id_y' in movies.columns:
    movies = movies.rename(columns={'id_y': 'id'})



In [4]:

def extract_names(x, n=None):
    try:
        x = ast.literal_eval(x)
        names = [i["name"] for i in x]
        return names[:n] if n else names
    except:
        return []

def extract_director(x):
    try:
        for i in ast.literal_eval(x):
            if i["job"] == "Director":
                return i["name"]
        return ""
    except:
        return ""

movies["genres"] = movies["genres"].apply(extract_names)
movies["keywords"] = movies["keywords"].apply(extract_names)
movies["cast"] = movies["cast"].apply(lambda x: extract_names(x, 3))
movies["director"] = movies["crew"].apply(extract_director)
movies["overview"] = movies["overview"].fillna("")
movies["release_year"] = movies["release_date"].str[:4]

movies["text"] = movies.apply(
    lambda x: " ".join([
        x["overview"],
        " ".join(x["genres"]),
        " ".join(x["keywords"]),
        " ".join(x["cast"]),
        x["director"]
    ]),
    axis=1
)

In [5]:
df = movies[["id", "title", "text", "genres", "vote_average", "release_year"]]
model = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = normalize(model.encode(df["text"].tolist(), batch_size=64, show_progress_bar=True))

index = faiss.IndexFlatIP(embeddings.shape[1])
index.add(embeddings.astype("float32"))



/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/11 [00:00<?, ?it/s]

In [6]:
TMDB_API_KEY = "9fd359fb33acbe9586c965c93d7e2e42"
IMG = "https://image.tmdb.org/t/p/w500"
poster_cache = {}
def get_poster(title):
    if title in poster_cache:
        return poster_cache[title]
    try:
        r = requests.get(
            "https://api.themoviedb.org/3/search/movie",
            params={"api_key": TMDB_API_KEY, "query": title},
            timeout=5
        ).json()
        if r.get("results"):
            path = r["results"][0].get("poster_path")
            if path:
                poster_cache[title] = IMG + path
                return poster_cache[title]
    except:
        pass
    return "https://via.placeholder.com/500x750?text=No+Poster"

In [7]:
liked_movies = deque(maxlen=5)
liked_movie_ids = set()

def get_liked_table():
    if not liked_movies:
        return [["No liked movies yet"]]
    titles = []
    for i in liked_movies:
        m = df[df["id"] == i].iloc[0]
        titles.append([f"❤️ {m['title']} ({m['release_year']})"])
    return titles

def build_gallery(indices):
    out = []
    for i in indices:
        m = df.iloc[i]
        heart = "❤️" if m["id"] in liked_movie_ids else "🤍"
        out.append((
            get_poster(m["title"]),
            f"{heart} {m['title']}\n⭐ {m['vote_average']} • {m['release_year']}\n🎭 {', '.join(m['genres'][:2])}"
        ))
    return out
def search_movies(query):
    if not query.strip():
        return []
    match = df[df["title"].str.lower().str.contains(query.lower(), na=False)]
    if not match.empty:
        qv = embeddings[match.index[0]].reshape(1, -1)
    else:
        qv = normalize(model.encode([query]))

    _, idx = index.search(qv, 18)
    return build_gallery(idx[0])

def home_movies():
    if not liked_movies:
        top = df.nlargest(18, "vote_average").index
        return build_gallery(top)

    idxs = [df[df["id"] == i].index[0] for i in liked_movies]
    avg = normalize(np.mean(embeddings[idxs], axis=0).reshape(1, -1))
    _, idx = index.search(avg, 25)

    final = [i for i in idx[0] if df.iloc[i]["id"] not in liked_movie_ids][:18]
    return build_gallery(final)


def toggle_like(evt: gr.SelectData, gallery):
    poster, caption = gallery[evt.index]
    title = caption.split("\n")[0].replace("❤️ ", "").replace("🤍 ", "")
    movie = df[df["title"] == title].iloc[0]
    mid = int(movie["id"])

    if mid in liked_movie_ids:
        liked_movie_ids.remove(mid)
        liked_movies.remove(mid)
    else:
        liked_movie_ids.add(mid)
        liked_movies.append(mid)

    return gallery, get_liked_table()

In [8]:

with gr.Blocks(theme=gr.themes.Base()) as app:

    gr.HTML(
    """
    <style>
        .watch-now-container {
            position: sticky;
            top: 14px;
            float: right;
            z-index: 9999;
        }
    </style>

    <div class="watch-now-container">
        <a href="https://moviebox.ph/" target="_blank">
            <button style="
                background-color: #c9e4ca;
                color: #000;
                border: none;
                padding: 10px 20px;
                border-radius: 10px;
                font-size: 16px;
                font-weight: 600;
                cursor: pointer;
            ">
                Watch Now
            </button>
        </a>
    </div>
    """
)



    gr.Markdown("# 🎬 Let's Watch")

    with gr.Row():
        gr.Markdown("### ❤️ Liked Movies")

    liked_table = gr.Dataframe(
        headers=["Movie"],
        datatype=["str"],
        value=get_liked_table(),
        interactive=False,
        wrap=True,
        row_count=5,
        col_count=(1, "fixed")
    )

    search = gr.Textbox(placeholder="Search movies", label="Search")

    with gr.Row():
        home_btn = gr.Button("🏠 Home")
        search_btn = gr.Button("🔍 Search")

    gallery = gr.Gallery(
        columns=6,
        height=520,
        object_fit="cover",
        show_download_button=False,
        preview=False
    )

    search_btn.click(search_movies, search, gallery)
    search.submit(search_movies, search, gallery)
    home_btn.click(home_movies, outputs=gallery)

    gallery.select(
        fn=toggle_like,
        inputs=gallery,
        outputs=[gallery, liked_table]
    )

    app.load(
        fn=lambda: (home_movies(), get_liked_table()),
        outputs=[gallery, liked_table]
    )

app.launch(debug=True)


/tmp/ipython-input-1088836903.py:1: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(theme=gr.themes.Base()) as app:


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://fcbf5018da4e65a598.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://fcbf5018da4e65a598.gradio.live


In [10]:
import nbformat

nb = nbformat.read("lets_watch.ipynb", as_version=4)

# Remove invalid widget metadata
if "widgets" in nb.metadata:
    del nb.metadata["widgets"]

# Save cleaned notebook
nbformat.write(nb, "lets_watch_github.ipynb")


FileNotFoundError: [Errno 2] No such file or directory: 'lets_watch.ipynb'